In [1]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes
!pip install nltk==3.7
!pip install -U Flask
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
import hashlib

nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [26]:
from flask import Flask, request, jsonify
import collections
import csv
import gzip

In [27]:
%cd /home/dataproc
# from inverted_index_gcp import InvertedIndex
# from inverted_index_gcp import *
# import inverted_index_gcp as iig
from inverted_index_gcp import *

/home/dataproc


In [4]:
# import importlib
# importlib.reload(iig)

In [4]:
path_anchor = "/home/dataproc/postings_gcp_anchor"
path_body = "/home/dataproc/postings_gcp_body"
path_title = "/home/dataproc/postings_gcp_title"

In [5]:
# Read indexes
body_index = InvertedIndex.read_index('.',"body_index")
anchor_index = InvertedIndex.read_index('.',"anchor_index")
title_index = InvertedIndex.read_index('.',"title_index")

In [6]:
body_index.bin_path = path_body
anchor_index.bin_path = path_anchor
title_index.bin_path = path_title

In [7]:
with open('doc_id_title_dict.pickle','rb') as f:
    doc_title_dict = pickle.load(f)
    
with open('doc_len_dict.pickle','rb') as f:
    doc_len_dict = pickle.load(f)
    
with open('page_views_dict.pkl','rb') as f:
    page_views_dict = pickle.load(f)

with open('page_rank_dict.pickle','rb') as f:
    page_rank_dict = pickle.load(f)

In [7]:
from contextlib import closing
def read_posting_list(inverted, w, index_type=""):
    with closing(MultiFileReader()) as reader:
        locs = inverted.posting_locs[w]
        b = reader.read(locs, inverted.df[w] * TUPLE_SIZE,index_type)
        posting_list = []
        for i in range(inverted.df[w]):
            doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
            tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
            posting_list.append((doc_id, tf))
        return posting_list

In [8]:
# Reading posting list example:
# lst = read_posting_list(body_index,"texas", path_body)
# print(len(lst))

In [9]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links",
                    "may", "first", "see", "history", "people", "one", "two",
                    "part", "thumb", "including", "second", "following",
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

def tokenize(text):
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    tokens = set(tok for tok in tokens if tok not in all_stopwords)
    return tokens



In [53]:
def search_title():
    ''' Returns ALL (not just top 100) search results that contain A QUERY WORD 
        IN THE TITLE of articles, ordered in descending order of the NUMBER OF 
        QUERY WORDS that appear in the title. For example, a document with a 
        title that matches two of the query words will be ranked before a 
        document with a title that matches only one query term. 

        Test this by navigating to the a URL like:
         http://YOUR_SERVER_DOMAIN/search_title?query=hello+world
        where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io
        if you're using ngrok on Colab or your external IP on GCP.
    Returns:
    --------
        list of ALL (not just top 100) search results, ordered from best to 
        worst where each element is a tuple (wiki_id, title).
    '''

    # query=["hello world"] - > [(wiki_id, title)] sorted decreasing
    # in a way that the title that contains most words of the query

    res = []
#     query = request.args.get('hello world', '')
    query = "LinkedIn"
#     query = "hello"
    if len(query) == 0:
        return jsonify(res)
    # BEGIN SOLUTION

    counter = collections.Counter()
    for token in tokenize(query):
        for docId_tf in title_index.read_posting_list(token):
            doc_id = docId_tf[0]
            title = doc_title_dict[doc_id]
            counter[(doc_id, title)] += 1

    
#     res = [map(lambda tup:tup[0], counter.most_common())] 
    res = list(map(lambda tup:tup[0], counter.most_common()))
    # END SOLUTION
    return res

In [54]:
print(search_title())

[(970755, 'LinkedIn'), (27769500, 'LinkedIn Pulse'), (36070366, '2012 LinkedIn hack'), (41726116, 'LinkedIn Learning'), (50191962, 'Timeline of LinkedIn'), (57147095, 'LinkedIn Top Companies'), (62976368, 'HiQ Labs v. LinkedIn')]


In [57]:
def search_anchor():
    # query=["hello world"] - > [(wiki_id, title)] sorted decreasing
    # in a way that the title that contains most words of the query

    res = []
#     query = request.args.get('hello world', '')
    query = "LinkedIn"
#     query = "hello"
    if len(query) == 0:
        return jsonify(res)
    # BEGIN SOLUTION

    counter_anchor = collections.Counter()
    for token in tokenize(query):
        for docId_tf in anchor_index.read_posting_list(token):
            doc_id = docId_tf[0]
            title = doc_title_dict[doc_id]
            counter_anchor[(doc_id, title)] += 1

    
#     res = [map(lambda tup:tup[0], counter.most_common())] 
    res = list(map(lambda tup:tup[0], counter_anchor.most_common()))
    # END SOLUTION
    return res

In [31]:
# print(search_anchor())

In [61]:
def get_pageview():
    ''' Returns the number of page views that each of the provide wiki articles
        had in August 2021.

        Test this by issuing a POST request to a URL like:
          http://YOUR_SERVER_DOMAIN/get_pageview
        with a json payload of the list of article ids. In python do:
          import requests
          requests.post('http://YOUR_SERVER_DOMAIN/get_pageview', json=[1,5,8])
        As before YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io
        if you're using ngrok on Colab or your external IP on GCP.
    Returns:
    --------
        list of ints:
          list of page view numbers from August 2021 that correrspond to the 
          provided list article IDs.
    '''

    res = []
#     wiki_ids = request.get_json()
    wiki_ids = [848, 21105323, 23536538, 23797577]
    if len(wiki_ids) == 0:
        return res
    # BEGIN SOLUTION
    
    res = [page_views_dict.get(doc_id, -1) for doc_id in wiki_ids]

    # END SOLUTION
    return res

In [62]:
print(get_pageview())

[84370, 1042, 30483, 38197]


In [34]:
def get_pagerank():
    ''' Returns PageRank values for a list of provided wiki article IDs. 

        Test this by issuing a POST request to a URL like:
          http://YOUR_SERVER_DOMAIN/get_pagerank
        with a json payload of the list of article ids. In python do:
          import requests
          requests.post('http://YOUR_SERVER_DOMAIN/get_pagerank', json=[1,5,8])
        As before YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io
        if you're using ngrok on Colab or your external IP on GCP.
    Returns:
    --------
        list of floats:
          list of PageRank scores that correrspond to the provided article IDs.
    '''
    res = []
#     wiki_ids = request.get_json()
    wiki_ids = [848, 3434750, 32927]
    if len(wiki_ids) == 0:
        return jsonify(res)
    # BEGIN SOLUTION

    res = [page_rank_dict.get(doc_id, -1) for doc_id in wiki_ids]
    # END SOLUTION
#     return jsonify(res)
    return res

In [ ]:
print(get_pagerank())

[50.7020900020747, 9913.728782160773, 5282.081575765278]
